# 1.Filter sequences that meet criteria

## Filter short sequences[27000-30000]

In [1]:
! seqkit stats 0.rawdata/guangdong.fasta

file                       format  type  num_seqs      sum_len  min_len   avg_len  max_len
0.rawdata/guangdong.fasta  FASTA   DNA      6,193  184,068,678   28,572  29,722.1   29,876


In [2]:
! wc -l 0.rawdata/guangdong_metadata.tsv

6194 0.rawdata/guangdong_metadata.tsv


In [3]:
# View date range
!cut -f 3 0.rawdata/guangdong_metadata.tsv | sort | uniq

2022/10/22
2022/10/23
2022/10/24
2022/10/25
2022/10/26
2022/10/30
2022/10/31
2022/11/1
2022/11/10
2022/11/11
2022/11/12
2022/11/13
2022/11/14
2022/11/15
2022/11/16
2022/11/17
2022/11/18
2022/11/19
2022/11/2
2022/11/20
2022/11/21
2022/11/22
2022/11/23
2022/11/24
2022/11/25
2022/11/26
2022/11/27
2022/11/28
2022/11/29
2022/11/3
2022/11/30
2022/11/4
2022/11/5
2022/11/6
2022/11/7
2022/11/8
2022/11/9
2022/12/1
2022/12/10
2022/12/11
2022/12/12
2022/12/13
2022/12/14
2022/12/15
2022/12/16
2022/12/17
2022/12/18
2022/12/19
2022/12/2
2022/12/20
2022/12/21
2022/12/22
2022/12/23
2022/12/24
2022/12/25
2022/12/26
2022/12/27
2022/12/28
2022/12/29
2022/12/3
2022/12/30
2022/12/31
2022/12/4
2022/12/5
2022/12/6
2022/12/7
2022/12/8
2022/12/9
2023/1/1
2023/1/10
2023/1/11
2023/1/12
2023/1/13
2023/1/14
2023/1/15
2023/1/16
2023/1/17
2023/1/18
2023/1/19
2023/1/2
2023/1/20
2023/1/21
2023/1/23
2023/1/24
2023/1/25
2023/1/26
2023/1/27
2023/1/28
2023/1/29
2023/1/3
2023/1/30
2023/1/4
2023/1/5
2023/1/6
2023/1/8
2023/1/

## Run nextclade pipeline

In [ ]:
! mkdir 2.nextclade
! nextclade run --input-dataset /home/qiuming/sars-cov-2-0128update --output-tsv=2.nextclade/guangdong_nextclade.tsv -j 48 0.rawdata/guangdong.fasta

## Filter sequences with unqualified quality and private mutations >= 10
[10]qc.overallStatus、[54]qc.privateMutations.status、[58]qc.snpClusters.status、[55]qc.totalprivate.number

In [5]:
# filter bad & private.mutatution.number >= 10
count = 0
with open('2.nextclade/guangdong_nextclade.tsv', 'r') as file, open('2.nextclade/guangdong_nextclade_bad.tsv', 'w') as f_out_bad, open('2.nextclade/guangdong_nextclade_good.tsv', 'w') as f_out_good:
    for line in file:
        if line.startswith('index'):
            continue
        columns = line.split('\t')
        
        if columns[9] == 'bad' or columns[53] == 'bad' or columns[57] == 'bad' or (not columns[54]) or (columns[54] and int(columns[54]) >= 10):
            count += 1
            f_out_bad.write(line)
        else:
            f_out_good.write(line)

print(count)


380


In [6]:
# check
! wc -l 2.nextclade/guangdong_nextclade_good.tsv
!cut -f 10 2.nextclade/guangdong_nextclade_good.tsv | sort | uniq
!cut -f 54 2.nextclade/guangdong_nextclade_good.tsv | sort | uniq
!cut -f 58 2.nextclade/guangdong_nextclade_good.tsv | sort | uniq

5813 2.nextclade/guangdong_nextclade_good.tsv
good
mediocre
good
good
mediocre


## get sequences.fasta

In [ ]:
! mkdir 3.filter
! cut -f 2 2.nextclade/guangdong_nextclade_good.tsv > 3.filter/guangdong_filtered_sequence_name.tsv
! seqkit grep -f 3.filter/guangdong_filtered_sequence_name.tsv 0.rawdata/guangdong.fasta > 3.filter/guangdong_filtered_sequence.fasta
! seqkit stats 3.filter/guangdong_filtered_sequence.fasta

# 2.Randomly select 500 sequences to construct a phylogenetic tree🌲

In [17]:
! mkdir 4.random_500
! seqkit sample -n 502 -o 4.random_500/guangdong_500.fasta 3.filter/filtered_sequence.fasta
! seqkit seq -n 4.random_500/guangdong_500.fasta > 4.random_500/random_500sequence_name.txt

## get metadata

In [23]:
import pandas as pd

seqMeta = pd.read_csv('1.withMetadata/seqMeta.tsv', sep='\t', header=None)
gd_no1row = pd.read_csv('2.nextclade/gd-no1row.tsv', sep='\t', header=None)

for index, row in seqMeta.iterrows():
    matching_rows = gd_no1row[gd_no1row[1] == row[0]]
    if not matching_rows.empty:
        seqMeta.at[index, 10] = matching_rows.iloc[0, 2]
        seqMeta.at[index, 11] = matching_rows.iloc[0, 3]

seqMeta[[0, 2, 5, 4, 8, 1, 6]].to_csv('1.withMetadata/seqMeta_fix', sep='\t', index=False, header=False)



In [24]:
# metadata
import pandas as pd

seqMeta = pd.read_csv('1.withMetadata/seqMeta_fix', sep='\t', header=None)
random_sequence = pd.read_csv('4.random_500/random_500sequence_name.txt', sep='\t', header=None)

merged_data = pd.merge(seqMeta, random_sequence, left_on=0, right_on=0)
merged_data.to_csv('4.random_500/guangdong_500_metadata.tsv', sep='\t', index=False, header=False)


# 3.calculate count&frequency

In [ ]:
! mkdir 5.clade
import pandas as pd

output_good = pd.read_csv('2.nextclade/guangdong_nextclade_good.tsv', sep='\t', header=None)
seqMeta = pd.read_csv('0.rawdata/guangdong_metadata.tsv', sep='\t', header=None)

output = pd.merge(output_good, seqMeta, left_on=1, right_on=0)
output = output[["0_y", "2_y", "2_x", "3_x"]]
output.to_csv('5.clade/guangdong_clade.tsv', sep='\t', index=False, header=False)


In [9]:
# view clade distribution
!cut -f 3 5.clade/guangdong_clade.tsv | sort | uniq

21L
22B
22D
22E
22F
23A
23B
23C
23D
23E
23F
23H
recombinant


### mark BA.5.2.48/BF.7

In [10]:
import pandas as pd
df = pd.read_csv('5.clade/guangdong_clade.tsv', sep='\t', header=None, index_col=None)

for index, row in df.iterrows():
    if row[2] == '22B':
        if row[3].startswith(('DY.1', 'DY.2', 'DY.3', 'DY.4', 'BA.5.2.48', 'DY.1.1')):
            df.at[index, 2] = '22B(BA.5.2.48)'
        elif row[3].startswith(('BF.7.14', 'BF.7.14.1', 'BF.7.14.2', 'BF.7.14.3', 'BF.7.14.4', 'BF.7.14.5', 'BF.7.14.6')):
            df.at[index, 2] = '22B(BF.7)'
        else:
            df.at[index, 2] = '22B(Other)'
df.to_csv("5.clade/guangdong_clade_marked.tsv", sep="\t", index=None, header=None)


In [11]:
! cut -f 3 5.clade/guangdong_clade_marked.tsv | sort | uniq -c

      4 21L
   2351 22B(BA.5.2.48)
    302 22B(BF.7)
      5 22B(Other)
     57 22D
      1 22E
    322 22F
    190 23A
    790 23B
      2 23C
   1383 23D
     98 23E
    295 23F
      1 23H
     12 recombinant


In [12]:
import pandas as pd
df = pd.read_csv('5.clade/guangdong_clade_marked.tsv', sep='\t', header=None, index_col=None)

############# number ###############
# Convert to datetime
df[1] = pd.to_datetime(df[1], format='%Y-%m-%d', errors='coerce')
df = df[(df[1] >= '2022-10-01') & (df[1] <= '2023-07-31')]

# Group by week and clade
weekly_clade_counts = df.groupby([pd.Grouper(key=1, freq='W'), 2]).size().unstack(fill_value=0)
weekly_clade_counts.to_csv('5.clade/guangdong2_weekly_clade_counts.csv')

############ frequency ################
weekly_clade_distribution = weekly_clade_counts.div(weekly_clade_counts.sum(axis=1), axis=0)
weekly_clade_distribution.to_csv('5.clade/guangdong2_weekly_clade_distribution.csv')